In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import seaborn
import matplotlib.pyplot as plt
import matplotlib
import re

In [3]:
%matplotlib inline

In [4]:
import pandas as pd 
import numpy as np
from collections import defaultdict

In [5]:
from tqdm import tqdm
from tqdm import tqdm_notebook

In [6]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
from sklearn.metrics import roc_curve

In [47]:
from model import *
from transformers import *
from quality import *
from save_load import *

In [10]:
train = pd.read_csv("ml5/train.csv", sep=";")
test = pd.read_csv("ml5/test.csv", sep=";")
test["smoke"] = test["smoke"].apply(lambda x: np.nan if (x == "None")  else int(x))
test["alco"] = test["alco"].apply(lambda x: np.nan if (x == "None")  else int(x))
test["active"] = test["active"].apply(lambda x: np.nan if (x == "None")  else int(x))

In [11]:
def clone_nans(df, features):
    for feature in features:
        df1 = df[~df[feature].isnull()].copy()
        df2 = df[df[feature].isnull()].copy()
        df3 = df2.copy()
        df2[feature] = np.zeros(df2.shape[0])
        df3[feature] = np.ones(df3.shape[0])
        df = pd.concat([df1, df2, df3])
        
    return df
test2 = clone_nans(test, ["smoke", "alco", "active"])

In [12]:
ml = Model(train, test2, "cardio")

In [13]:
def ap_lo_func(x, axis):
    if x < 0:
        return -x
    if x == 0:
        return 40
    elif x <= 20:
        return 10 * x
    elif x <= 300:
        return x
    elif x <= 2060:
        return x * 1.0 / 10
    elif x <= 2300:
        return x - 2000
    else:
        return x * 1.0 / 100

In [14]:
def ap_hi_func(x, axis):
    if x < 0:
        return - x
    elif x == 0:
        return 120
    elif x <= 2:
        return x * 100
    elif x <= 20:
        return x * 10
    elif x <= 500:
        return x
    elif x <= 2060:
        return x * 1.0 / 10
    elif x <= 2300:
        return x - 2000
    else:
        return x * 1.0 / 100

In [15]:
def days_to_year(x, axis):
    return x * 1.0 / 365

In [16]:
class swap_ap_transformer():
    def __init__(self, tag, ap_lo = "aplopr", ap_hi = "aphipr", ap_lo_target = "aplo", ap_hi_target = "aphi"):
        self.ap_lo = "aplopr"
        self.ap_hi = "aphipr"
        self.tag = tag
        
        self.ap_lo_target = ap_lo_target
        self.ap_hi_target = ap_hi_target
    
    def fit_transform(self, train):
        train[self.ap_lo_target] = train[[self.ap_lo, self.ap_hi]].min(axis = 1)
        train[self.ap_hi_target] = train[[self.ap_lo, self.ap_hi]].max(axis = 1)
        
        return train
    
    def transform(self, test):
        test[self.ap_lo_target] = test[[self.ap_lo, self.ap_hi]].min(axis = 1)
        test[self.ap_hi_target] = test[[self.ap_lo, self.ap_hi]].max(axis = 1)
        
        return test
        
    def get_column_list(self):
        return [self.ap_lo_target, self.ap_hi_target]

In [18]:
ml.remove_step("aplopr")
ml.add_step("aplopr", custom_transformer(ap_lo_func, ["ap_lo"], "aplopr"))
ml.remove_step("aphipr")
ml.add_step("aphipr", custom_transformer(ap_hi_func, ["ap_hi"], "aphipr"))
ml.remove_step("year")
ml.add_step("year", custom_transformer(days_to_year, ["age"], "year"))
ml.remove_step("swap_ap")
ml.add_step("swap_ap", swap_ap_transformer("swap_ap"), prerequisites = ["aplopr", "aphipr"])


ml.compute_step("aplopr")
ml.compute_step("aphipr")
ml.compute_step("year")
ml.compute_step("swap_ap")

Trying to remove undefined step swap_ap.


In [19]:
ml.remove_step("onehot")
ml.add_step("onehot", onehot_transformer(["cholesterol", "gluc"], "onehot"))
ml.compute_step("onehot")

Trying to remove undefined step onehot.


In [34]:
ml.remove_step("log")
ml.add_step("log", logarithmic_transformer(["year", "height", "weight", "aphi", "aplo"], 0, "log"), prerequisites = ["year", "aphipr", "aplopr"])
ml.compute_step("log")

In [35]:
ml.train.columns

Index([u'id', u'age', u'gender', u'height', u'weight', u'ap_hi', u'ap_lo',
       u'cholesterol', u'gluc', u'smoke', u'alco', u'active', u'cardio',
       u'aplopr', u'aphipr', u'year', u'aplo', u'aphi', u'cholesterol_0',
       u'cholesterol_1', u'cholesterol_2', u'gluc_3', u'gluc_4', u'gluc_5',
       u'log_aplo', u'log_height', u'log_weight', u'log_aphi', u'log_year'],
      dtype='object')

In [36]:
fl_n1 = ["year", "height", "weight", "aphi", "aplo", "log_aplo", "log_height", "log_weight", "log_aphi", "log_year"]
fl_c1 = ["gender", "cholesterol_0", "cholesterol_1", "cholesterol_2", "gluc_3", "gluc_4", "gluc_5", "alco", "smoke", "active"]

In [37]:
ml.remove_step("mulf1")
ml.add_step("mulf1", multiplication_transformer(fl_n1, [-2, -1, 2], 1, "mulf1"), prerequisites = ["year", "aphipr", "aplopr"])

In [38]:
ml.remove_step("mulf2")
ml.add_step("mulf2", multiplication_transformer(fl_n1, [-2, -1, 1, 2], 2, "mulf2"), prerequisites = ["year", "aphipr", "aplopr"])

In [39]:
ml.remove_step("mulf3")
ml.add_step("mulf3", multiplication_transformer(fl_n1, [-1, 1], 3, "mulf3"), prerequisites = ["year", "aphipr", "aplopr"])

In [40]:
ml.compute_step("mulf1")

In [41]:
ml.compute_step("mulf2")

In [42]:
ml.compute_step("mulf3")

In [43]:
fl1 = fl_n1 + fl_c1 + ml.get_step_columns(["mulf1", "mulf2", "mulf3"])
fl2 = fl_n1 + fl_c1 + ml.get_step_columns(["mulf1", "mulf2"])
fl3 = fl_n1 + fl_c1 + ml.get_step_columns(["mulf1"])

In [44]:
print len(fl1)
print len(fl2)
print len(fl3)

1730
770
50


In [45]:
ml.remove_step("scale1")
ml.add_step("scale1", scaling_transformer(fl1, "scale1"), prerequisites = ["mulf1", "mulf2", "mulf3"])
ml.compute_step("scale1")

Trying to remove undefined step scale1.


In [48]:
ml.remove_step("pca50")
ml.add_step("pca50", PCA_transformer(ml.get_step_columns("scale1"), 50, "pca50"), prerequisites = ["scale1"])

Trying to remove undefined step pca50.


In [49]:
ml.compute_step("pca50")

In [50]:
lrclf_params_1 = {"C": 0.2, "n_jobs" : -1, "penalty": "l1"}
ml.remove_step("lr_s1")
ml.add_step("lr_s1", logreg_stacking(ml.get_step_columns("pca50"), "cardio", lrclf_params_1, 10, "lr_s1"), prerequisites = ["scale1"])
ml.compute_step("lr_s1")

Trying to remove undefined step lr_s1.


In [51]:
measure_logloss(ml, "train", "cardio", "lr_s1")

0.54980445539757594

In [52]:
lrclf_params_1 = {"C": 0.2, "n_jobs" : -1, "penalty": "l1"}
ml.remove_step("lr_s2")
ml.add_step("lr_s2", logreg_stacking(ml.get_step_columns("scale1"), "cardio", lrclf_params_1, 10, "lr_s2"), prerequisites = ["scale1"])
ml.compute_step("lr_s2")

Trying to remove undefined step lr_s2.


In [53]:
measure_logloss(ml, "train", "cardio", "lr_s2")

0.54971855500207167

In [54]:
#xgbclf_params = {"objective":"binary:logistic", "max_depth":5, "learning_rate":0.01,
#                 "n_estimators": 500, "nthread":8, "base_score":0.5, "seed":47, "missing":None}
#ml.remove_step("xgb_s1")
#ml.add_step("xgb_s1", xgb_stacking(ml.get_step_columns("scale1"), "cardio", xgbclf_params, 5, "xgb_s1"), prerequisites = ["scale1"])
#ml.compute_step("xgb_s1")
#measure_logloss(ml, "train", "cardio", "xgb_s1")

In [55]:
lgbmclf_params = {"objective":"binary", "max_depth":7, "learning_rate":0.01,
                 "n_estimators": 500, "nthread":8, "seed":47, "feature_fraction":0.8}
ml.remove_step("lgbm_s1")
ml.add_step("lgbm_s1", lgbm_stacking(ml.get_step_columns(["scale1", "pca50"]), "cardio", lgbmclf_params, 10, "lgbm_s1"), prerequisites = ["scale1"])

Trying to remove undefined step lgbm_s1.


In [56]:
ml.compute_step("lgbm_s1")

[1]	training's binary_logloss: 0.690358	valid_1's binary_logloss: 0.690452
[2]	training's binary_logloss: 0.687617	valid_1's binary_logloss: 0.687812
[3]	training's binary_logloss: 0.684925	valid_1's binary_logloss: 0.685216
[4]	training's binary_logloss: 0.682294	valid_1's binary_logloss: 0.682675
[5]	training's binary_logloss: 0.679712	valid_1's binary_logloss: 0.680173
[6]	training's binary_logloss: 0.677177	valid_1's binary_logloss: 0.677716
[7]	training's binary_logloss: 0.674685	valid_1's binary_logloss: 0.675317
[8]	training's binary_logloss: 0.672242	valid_1's binary_logloss: 0.672959
[9]	training's binary_logloss: 0.669845	valid_1's binary_logloss: 0.670653
[10]	training's binary_logloss: 0.667494	valid_1's binary_logloss: 0.668393
[11]	training's binary_logloss: 0.66519	valid_1's binary_logloss: 0.666186
[12]	training's binary_logloss: 0.662925	valid_1's binary_logloss: 0.664019
[13]	training's binary_logloss: 0.660704	valid_1's binary_logloss: 0.661888
[14]	training's binary

[110]	training's binary_logloss: 0.557593	valid_1's binary_logloss: 0.566131
[111]	training's binary_logloss: 0.55716	valid_1's binary_logloss: 0.565765
[112]	training's binary_logloss: 0.556732	valid_1's binary_logloss: 0.565398
[113]	training's binary_logloss: 0.556307	valid_1's binary_logloss: 0.565038
[114]	training's binary_logloss: 0.555887	valid_1's binary_logloss: 0.564684
[115]	training's binary_logloss: 0.55548	valid_1's binary_logloss: 0.564339
[116]	training's binary_logloss: 0.555071	valid_1's binary_logloss: 0.563998
[117]	training's binary_logloss: 0.554672	valid_1's binary_logloss: 0.563664
[118]	training's binary_logloss: 0.554274	valid_1's binary_logloss: 0.56333
[119]	training's binary_logloss: 0.55389	valid_1's binary_logloss: 0.563023
[120]	training's binary_logloss: 0.553515	valid_1's binary_logloss: 0.562707
[121]	training's binary_logloss: 0.553137	valid_1's binary_logloss: 0.562395
[122]	training's binary_logloss: 0.552766	valid_1's binary_logloss: 0.562094
[12

[218]	training's binary_logloss: 0.533049	valid_1's binary_logloss: 0.548889
[219]	training's binary_logloss: 0.532942	valid_1's binary_logloss: 0.548851
[220]	training's binary_logloss: 0.53283	valid_1's binary_logloss: 0.548811
[221]	training's binary_logloss: 0.532722	valid_1's binary_logloss: 0.548768
[222]	training's binary_logloss: 0.532617	valid_1's binary_logloss: 0.548712
[223]	training's binary_logloss: 0.532504	valid_1's binary_logloss: 0.548672
[224]	training's binary_logloss: 0.532398	valid_1's binary_logloss: 0.548623
[225]	training's binary_logloss: 0.532293	valid_1's binary_logloss: 0.548584
[226]	training's binary_logloss: 0.532191	valid_1's binary_logloss: 0.548542
[227]	training's binary_logloss: 0.532088	valid_1's binary_logloss: 0.548512
[228]	training's binary_logloss: 0.531989	valid_1's binary_logloss: 0.548468
[229]	training's binary_logloss: 0.531885	valid_1's binary_logloss: 0.548421
[230]	training's binary_logloss: 0.531789	valid_1's binary_logloss: 0.548372


[325]	training's binary_logloss: 0.524807	valid_1's binary_logloss: 0.546793
[326]	training's binary_logloss: 0.524746	valid_1's binary_logloss: 0.546796
[327]	training's binary_logloss: 0.524685	valid_1's binary_logloss: 0.546803
[328]	training's binary_logloss: 0.524627	valid_1's binary_logloss: 0.546809
[329]	training's binary_logloss: 0.524568	valid_1's binary_logloss: 0.546797
[330]	training's binary_logloss: 0.524507	valid_1's binary_logloss: 0.546794
[331]	training's binary_logloss: 0.524449	valid_1's binary_logloss: 0.546783
[332]	training's binary_logloss: 0.524387	valid_1's binary_logloss: 0.546785
[333]	training's binary_logloss: 0.524327	valid_1's binary_logloss: 0.546776
[334]	training's binary_logloss: 0.524268	valid_1's binary_logloss: 0.546786
[335]	training's binary_logloss: 0.524207	valid_1's binary_logloss: 0.546788
[336]	training's binary_logloss: 0.524148	valid_1's binary_logloss: 0.546786
[337]	training's binary_logloss: 0.524091	valid_1's binary_logloss: 0.546774

[433]	training's binary_logloss: 0.519129	valid_1's binary_logloss: 0.547002
[434]	training's binary_logloss: 0.51908	valid_1's binary_logloss: 0.547019
[435]	training's binary_logloss: 0.519031	valid_1's binary_logloss: 0.547006
[436]	training's binary_logloss: 0.518983	valid_1's binary_logloss: 0.546999
[437]	training's binary_logloss: 0.518934	valid_1's binary_logloss: 0.547005
[438]	training's binary_logloss: 0.518887	valid_1's binary_logloss: 0.547013
[439]	training's binary_logloss: 0.518839	valid_1's binary_logloss: 0.547017
[440]	training's binary_logloss: 0.51879	valid_1's binary_logloss: 0.547011
[441]	training's binary_logloss: 0.518743	valid_1's binary_logloss: 0.547011
[442]	training's binary_logloss: 0.518696	valid_1's binary_logloss: 0.546999
[443]	training's binary_logloss: 0.51865	valid_1's binary_logloss: 0.547015
[444]	training's binary_logloss: 0.518614	valid_1's binary_logloss: 0.547021
[445]	training's binary_logloss: 0.51857	valid_1's binary_logloss: 0.547015
[44

...

[249]	training's binary_logloss: 0.530875	valid_1's binary_logloss: 0.540736
[250]	training's binary_logloss: 0.530792	valid_1's binary_logloss: 0.54069
[251]	training's binary_logloss: 0.530708	valid_1's binary_logloss: 0.540654
[252]	training's binary_logloss: 0.530624	valid_1's binary_logloss: 0.540624
[253]	training's binary_logloss: 0.530543	valid_1's binary_logloss: 0.540593
[254]	training's binary_logloss: 0.53046	valid_1's binary_logloss: 0.540564
[255]	training's binary_logloss: 0.530375	valid_1's binary_logloss: 0.540538
[256]	training's binary_logloss: 0.530293	valid_1's binary_logloss: 0.540488
[257]	training's binary_logloss: 0.530213	valid_1's binary_logloss: 0.540446
[258]	training's binary_logloss: 0.530134	valid_1's binary_logloss: 0.540395
[259]	training's binary_logloss: 0.530056	valid_1's binary_logloss: 0.540362
[260]	training's binary_logloss: 0.529978	valid_1's binary_logloss: 0.540332
[261]	training's binary_logloss: 0.529901	valid_1's binary_logloss: 0.540297
[

[356]	training's binary_logloss: 0.523949	valid_1's binary_logloss: 0.538792
[357]	training's binary_logloss: 0.523896	valid_1's binary_logloss: 0.538783
[358]	training's binary_logloss: 0.523841	valid_1's binary_logloss: 0.538785
[359]	training's binary_logloss: 0.523787	valid_1's binary_logloss: 0.538787
[360]	training's binary_logloss: 0.523736	valid_1's binary_logloss: 0.538787
[361]	training's binary_logloss: 0.523684	valid_1's binary_logloss: 0.538782
[362]	training's binary_logloss: 0.52363	valid_1's binary_logloss: 0.538778
[363]	training's binary_logloss: 0.523579	valid_1's binary_logloss: 0.538771
[364]	training's binary_logloss: 0.523522	valid_1's binary_logloss: 0.538762
[365]	training's binary_logloss: 0.523471	valid_1's binary_logloss: 0.538756
[366]	training's binary_logloss: 0.523419	valid_1's binary_logloss: 0.538751
[367]	training's binary_logloss: 0.523368	valid_1's binary_logloss: 0.53874
[368]	training's binary_logloss: 0.523318	valid_1's binary_logloss: 0.538734
[

[464]	training's binary_logloss: 0.518666	valid_1's binary_logloss: 0.538599
[465]	training's binary_logloss: 0.518621	valid_1's binary_logloss: 0.538607
[466]	training's binary_logloss: 0.518578	valid_1's binary_logloss: 0.538603
[467]	training's binary_logloss: 0.518529	valid_1's binary_logloss: 0.538617
[468]	training's binary_logloss: 0.518476	valid_1's binary_logloss: 0.538625
[469]	training's binary_logloss: 0.518431	valid_1's binary_logloss: 0.538635
[470]	training's binary_logloss: 0.518387	valid_1's binary_logloss: 0.538623
[471]	training's binary_logloss: 0.518336	valid_1's binary_logloss: 0.53862
[472]	training's binary_logloss: 0.51829	valid_1's binary_logloss: 0.538619
[473]	training's binary_logloss: 0.518248	valid_1's binary_logloss: 0.538632
[474]	training's binary_logloss: 0.518207	valid_1's binary_logloss: 0.538646
[475]	training's binary_logloss: 0.518165	valid_1's binary_logloss: 0.538645
[476]	training's binary_logloss: 0.51812	valid_1's binary_logloss: 0.538645
[4

[135]	valid_0's binary_logloss: 0.549591
[136]	valid_0's binary_logloss: 0.549298
[137]	valid_0's binary_logloss: 0.549006
[138]	valid_0's binary_logloss: 0.548721
[139]	valid_0's binary_logloss: 0.548438
[140]	valid_0's binary_logloss: 0.548154
[141]	valid_0's binary_logloss: 0.54788
[142]	valid_0's binary_logloss: 0.547607
[143]	valid_0's binary_logloss: 0.547339
[144]	valid_0's binary_logloss: 0.547076
[145]	valid_0's binary_logloss: 0.546817
[146]	valid_0's binary_logloss: 0.546566
[147]	valid_0's binary_logloss: 0.546313
[148]	valid_0's binary_logloss: 0.546064
[149]	valid_0's binary_logloss: 0.545816
[150]	valid_0's binary_logloss: 0.545574
[151]	valid_0's binary_logloss: 0.545334
[152]	valid_0's binary_logloss: 0.545098
[153]	valid_0's binary_logloss: 0.544866
[154]	valid_0's binary_logloss: 0.544635
[155]	valid_0's binary_logloss: 0.544406
[156]	valid_0's binary_logloss: 0.544184
[157]	valid_0's binary_logloss: 0.543963
[158]	valid_0's binary_logloss: 0.543748
[159]	valid_0's b

[336]	valid_0's binary_logloss: 0.526342
[337]	valid_0's binary_logloss: 0.526288
[338]	valid_0's binary_logloss: 0.526237
[339]	valid_0's binary_logloss: 0.526187
[340]	valid_0's binary_logloss: 0.526135
[341]	valid_0's binary_logloss: 0.526085
[342]	valid_0's binary_logloss: 0.526034
[343]	valid_0's binary_logloss: 0.525981
[344]	valid_0's binary_logloss: 0.525933
[345]	valid_0's binary_logloss: 0.525884
[346]	valid_0's binary_logloss: 0.525835
[347]	valid_0's binary_logloss: 0.525787
[348]	valid_0's binary_logloss: 0.525735
[349]	valid_0's binary_logloss: 0.525689
[350]	valid_0's binary_logloss: 0.525636
[351]	valid_0's binary_logloss: 0.525588
[352]	valid_0's binary_logloss: 0.525538
[353]	valid_0's binary_logloss: 0.525488
[354]	valid_0's binary_logloss: 0.525434
[355]	valid_0's binary_logloss: 0.525381
[356]	valid_0's binary_logloss: 0.52533
[357]	valid_0's binary_logloss: 0.525284
[358]	valid_0's binary_logloss: 0.525232
[359]	valid_0's binary_logloss: 0.525184
[360]	valid_0's b

In [57]:
measure_logloss(ml, "train", "cardio", "lgbm_s1")

0.53945302904419279

In [58]:
#save_model(ml, "models/train1/")

In [59]:
#fig, ax = plt.subplots(1,1,figsize=(20,100))
#lightgbm.plot_importance(ml.EG.transformer_dict["lgbm_s1"].transformer.booster_, ax = ax)

In [60]:
x = zip(ml.EG.transformer_dict["lgbm_s1"].transformer.booster_.feature_importance(), ml.EG.transformer_dict["lgbm_s1"].transformer.booster_.feature_name())
lgbm_sortedfl = [k[1] for k in sorted(x, key = lambda x : -x[0])]

In [61]:
ml.remove_step("add2f")
ml.add_step("add2f", additive_featuring(lgbm_sortedfl[0:60], 2, "cardio", lrclf_params_1, "add2f"), prerequisites = ["scale1", "pca50"])
ml.compute_step("add2f")

Trying to remove undefined step add2f.


In [62]:
lgbmclf_params = {"objective":"binary", "max_depth":7, "learning_rate":0.01,
                 "n_estimators": 500, "nthread":8, "seed":47, "feature_fraction":0.8}
ml.remove_step("lgbm_s2")
ml.add_step("lgbm_s2", lgbm_stacking(ml.get_step_columns(["scale1", "pca50", "add2f", "lr_s2", "lgbm_s1"]), "cardio", lgbmclf_params, 10, "lgbm_s2"), prerequisites = ["scale1", "pca50", "add2f", "lr_s2"])
ml.compute_step("lgbm_s2")

Trying to remove undefined step lgbm_s2.


[1]	training's binary_logloss: 0.690288	valid_1's binary_logloss: 0.690426
[2]	training's binary_logloss: 0.687514	valid_1's binary_logloss: 0.68781
[3]	training's binary_logloss: 0.684764	valid_1's binary_logloss: 0.685201
[4]	training's binary_logloss: 0.68207	valid_1's binary_logloss: 0.682635
[5]	training's binary_logloss: 0.679454	valid_1's binary_logloss: 0.680162
[6]	training's binary_logloss: 0.676883	valid_1's binary_logloss: 0.67775
[7]	training's binary_logloss: 0.674339	valid_1's binary_logloss: 0.675357
[8]	training's binary_logloss: 0.671845	valid_1's binary_logloss: 0.672999
[9]	training's binary_logloss: 0.669398	valid_1's binary_logloss: 0.670676
[10]	training's binary_logloss: 0.666998	valid_1's binary_logloss: 0.66841
[11]	training's binary_logloss: 0.664663	valid_1's binary_logloss: 0.666205
[12]	training's binary_logloss: 0.662369	valid_1's binary_logloss: 0.664068
[13]	training's binary_logloss: 0.6601	valid_1's binary_logloss: 0.661927
[14]	training's binary_logl

[110]	training's binary_logloss: 0.555043	valid_1's binary_logloss: 0.566212
[111]	training's binary_logloss: 0.554598	valid_1's binary_logloss: 0.56587
[112]	training's binary_logloss: 0.554164	valid_1's binary_logloss: 0.565517
[113]	training's binary_logloss: 0.553736	valid_1's binary_logloss: 0.565156
[114]	training's binary_logloss: 0.553317	valid_1's binary_logloss: 0.564827
[115]	training's binary_logloss: 0.552907	valid_1's binary_logloss: 0.564485
[116]	training's binary_logloss: 0.552504	valid_1's binary_logloss: 0.564145
[117]	training's binary_logloss: 0.552102	valid_1's binary_logloss: 0.563829
[118]	training's binary_logloss: 0.551705	valid_1's binary_logloss: 0.563518
[119]	training's binary_logloss: 0.551311	valid_1's binary_logloss: 0.563216
[120]	training's binary_logloss: 0.550926	valid_1's binary_logloss: 0.562912
[121]	training's binary_logloss: 0.550544	valid_1's binary_logloss: 0.562617
[122]	training's binary_logloss: 0.550173	valid_1's binary_logloss: 0.562313


[217]	training's binary_logloss: 0.530439	valid_1's binary_logloss: 0.549316
[218]	training's binary_logloss: 0.530325	valid_1's binary_logloss: 0.549266
[219]	training's binary_logloss: 0.530212	valid_1's binary_logloss: 0.549225
[220]	training's binary_logloss: 0.530096	valid_1's binary_logloss: 0.549161
[221]	training's binary_logloss: 0.529988	valid_1's binary_logloss: 0.549111
[222]	training's binary_logloss: 0.529877	valid_1's binary_logloss: 0.54906
[223]	training's binary_logloss: 0.529764	valid_1's binary_logloss: 0.549017
[224]	training's binary_logloss: 0.529658	valid_1's binary_logloss: 0.548985
[225]	training's binary_logloss: 0.529551	valid_1's binary_logloss: 0.548943
[226]	training's binary_logloss: 0.529448	valid_1's binary_logloss: 0.548909
[227]	training's binary_logloss: 0.529341	valid_1's binary_logloss: 0.548865
[228]	training's binary_logloss: 0.529239	valid_1's binary_logloss: 0.548832
[229]	training's binary_logloss: 0.529132	valid_1's binary_logloss: 0.548799


[324]	training's binary_logloss: 0.521645	valid_1's binary_logloss: 0.547224
[325]	training's binary_logloss: 0.52158	valid_1's binary_logloss: 0.547218
[326]	training's binary_logloss: 0.521516	valid_1's binary_logloss: 0.547214
[327]	training's binary_logloss: 0.521455	valid_1's binary_logloss: 0.547203
[328]	training's binary_logloss: 0.521388	valid_1's binary_logloss: 0.547198
[329]	training's binary_logloss: 0.521324	valid_1's binary_logloss: 0.547187
[330]	training's binary_logloss: 0.521262	valid_1's binary_logloss: 0.547178
[331]	training's binary_logloss: 0.521194	valid_1's binary_logloss: 0.547166
[332]	training's binary_logloss: 0.521132	valid_1's binary_logloss: 0.547171
[333]	training's binary_logloss: 0.521065	valid_1's binary_logloss: 0.547161
[334]	training's binary_logloss: 0.521005	valid_1's binary_logloss: 0.547147
[335]	training's binary_logloss: 0.520939	valid_1's binary_logloss: 0.547155
[336]	training's binary_logloss: 0.520876	valid_1's binary_logloss: 0.547148


...

[333]	training's binary_logloss: 0.522526	valid_1's binary_logloss: 0.536253
[334]	training's binary_logloss: 0.52246	valid_1's binary_logloss: 0.536243
[335]	training's binary_logloss: 0.522398	valid_1's binary_logloss: 0.536237
[336]	training's binary_logloss: 0.522335	valid_1's binary_logloss: 0.536221
[337]	training's binary_logloss: 0.522275	valid_1's binary_logloss: 0.536208
[338]	training's binary_logloss: 0.522216	valid_1's binary_logloss: 0.536205
[339]	training's binary_logloss: 0.522156	valid_1's binary_logloss: 0.536196
[340]	training's binary_logloss: 0.522094	valid_1's binary_logloss: 0.536178
[341]	training's binary_logloss: 0.522038	valid_1's binary_logloss: 0.536161
[342]	training's binary_logloss: 0.521977	valid_1's binary_logloss: 0.536146
[343]	training's binary_logloss: 0.521916	valid_1's binary_logloss: 0.536145
[344]	training's binary_logloss: 0.521862	valid_1's binary_logloss: 0.536142
[345]	training's binary_logloss: 0.521804	valid_1's binary_logloss: 0.536136


[440]	training's binary_logloss: 0.516492	valid_1's binary_logloss: 0.535853
[441]	training's binary_logloss: 0.516433	valid_1's binary_logloss: 0.535849
[442]	training's binary_logloss: 0.516381	valid_1's binary_logloss: 0.535843
[443]	training's binary_logloss: 0.516327	valid_1's binary_logloss: 0.535834
[444]	training's binary_logloss: 0.516269	valid_1's binary_logloss: 0.535842
[445]	training's binary_logloss: 0.516218	valid_1's binary_logloss: 0.535845
[446]	training's binary_logloss: 0.51617	valid_1's binary_logloss: 0.535845
[447]	training's binary_logloss: 0.516117	valid_1's binary_logloss: 0.535849
[448]	training's binary_logloss: 0.516062	valid_1's binary_logloss: 0.535861
[449]	training's binary_logloss: 0.516017	valid_1's binary_logloss: 0.535857
[450]	training's binary_logloss: 0.515967	valid_1's binary_logloss: 0.535858
[451]	training's binary_logloss: 0.515915	valid_1's binary_logloss: 0.535852
[452]	training's binary_logloss: 0.51586	valid_1's binary_logloss: 0.535852
[

[90]	valid_0's binary_logloss: 0.566549
[91]	valid_0's binary_logloss: 0.565956
[92]	valid_0's binary_logloss: 0.56537
[93]	valid_0's binary_logloss: 0.564797
[94]	valid_0's binary_logloss: 0.564236
[95]	valid_0's binary_logloss: 0.56368
[96]	valid_0's binary_logloss: 0.563133
[97]	valid_0's binary_logloss: 0.562587
[98]	valid_0's binary_logloss: 0.562055
[99]	valid_0's binary_logloss: 0.561532
[100]	valid_0's binary_logloss: 0.561016
[101]	valid_0's binary_logloss: 0.560506
[102]	valid_0's binary_logloss: 0.560009
[103]	valid_0's binary_logloss: 0.559519
[104]	valid_0's binary_logloss: 0.559035
[105]	valid_0's binary_logloss: 0.558561
[106]	valid_0's binary_logloss: 0.558093
[107]	valid_0's binary_logloss: 0.557631
[108]	valid_0's binary_logloss: 0.557178
[109]	valid_0's binary_logloss: 0.556735
[110]	valid_0's binary_logloss: 0.556297
[111]	valid_0's binary_logloss: 0.555867
[112]	valid_0's binary_logloss: 0.555446
[113]	valid_0's binary_logloss: 0.555028
[114]	valid_0's binary_loglo

[291]	valid_0's binary_logloss: 0.526195
[292]	valid_0's binary_logloss: 0.526128
[293]	valid_0's binary_logloss: 0.526065
[294]	valid_0's binary_logloss: 0.526001
[295]	valid_0's binary_logloss: 0.525937
[296]	valid_0's binary_logloss: 0.525871
[297]	valid_0's binary_logloss: 0.525806
[298]	valid_0's binary_logloss: 0.525738
[299]	valid_0's binary_logloss: 0.525672
[300]	valid_0's binary_logloss: 0.525604
[301]	valid_0's binary_logloss: 0.525538
[302]	valid_0's binary_logloss: 0.525475
[303]	valid_0's binary_logloss: 0.525412
[304]	valid_0's binary_logloss: 0.525344
[305]	valid_0's binary_logloss: 0.525277
[306]	valid_0's binary_logloss: 0.525215
[307]	valid_0's binary_logloss: 0.525149
[308]	valid_0's binary_logloss: 0.525083
[309]	valid_0's binary_logloss: 0.52502
[310]	valid_0's binary_logloss: 0.524959
[311]	valid_0's binary_logloss: 0.524894
[312]	valid_0's binary_logloss: 0.52483
[313]	valid_0's binary_logloss: 0.524766
[314]	valid_0's binary_logloss: 0.524701
[315]	valid_0's bi

[492]	valid_0's binary_logloss: 0.515559
[493]	valid_0's binary_logloss: 0.515509
[494]	valid_0's binary_logloss: 0.515463
[495]	valid_0's binary_logloss: 0.515424
[496]	valid_0's binary_logloss: 0.515375
[497]	valid_0's binary_logloss: 0.515351
[498]	valid_0's binary_logloss: 0.515306
[499]	valid_0's binary_logloss: 0.515263
[500]	valid_0's binary_logloss: 0.515222


In [63]:
measure_logloss(ml, "train", "cardio", "lgbm_s2")

0.54001751939987819

In [65]:
lrclf_params_2 = {"C": 0.2, "n_jobs" : -1, "penalty": "l1"}
ml.remove_step("lr_s3")
ml.add_step("lr_s3", logreg_stacking(ml.get_step_columns(["scale1", "pca50", "add2f", "lr_s2", "lgbm_s1"]), "cardio", lrclf_params_2, 10, "lr_s3"), prerequisites = ["scale1", "pca50", "add2f", "lr_s2"])
ml.compute_step("lr_s3")

Trying to remove undefined step lr_s3.


In [66]:
measure_logloss(ml, "train", "cardio", "lr_s3")

0.54007898852644709

In [67]:
x2 = zip(ml.EG.transformer_dict["lgbm_s2"].transformer.booster_.feature_importance(), ml.EG.transformer_dict["lgbm_s2"].transformer.booster_.feature_name())
lgbm_sortedfl2 = [k[1] for k in sorted(x2, key = lambda x : -x[0])]

In [68]:
lgbm_sortedfl2

[u'lgbm_s1',
 u'lr_s2',
 u'add2f_342',
 u'add2f_278',
 u'add2f_117',
 u'add2f_645',
 u'add2f_253',
 u'add2f_109',
 u'add2f_451',
 u'add2f_569',
 u'add2f_127',
 u'add2f_52',
 u'add2f_1612',
 u'add2f_110',
 u'add2f_445',
 u'add2f_142',
 u'add2f_531',
 u'add2f_456',
 u'add2f_81',
 u'add2f_49',
 u'add2f_61',
 u'add2f_9',
 u'add2f_1329',
 u'add2f_138',
 u'add2f_767',
 u'add2f_35',
 u'add2f_784',
 u'add2f_277',
 u'add2f_63',
 u'add2f_241',
 u'add2f_712',
 u'add2f_3',
 u'add2f_513',
 u'add2f_664',
 u'add2f_139',
 u'add2f_783',
 u'add2f_236',
 u'add2f_1121',
 u'add2f_556',
 u'add2f_736',
 u'add2f_182',
 u'add2f_220',
 u'add2f_107',
 u'add2f_57',
 u'add2f_514',
 u'add2f_392',
 u'add2f_1578',
 u'add2f_1664',
 u'add2f_648',
 u'add2f_937',
 u'scale1_mulf3_932',
 u'add2f_59',
 u'add2f_60',
 u'pca50_25',
 u'add2f_251',
 u'add2f_303',
 u'add2f_914',
 u'add2f_156',
 u'add2f_619',
 u'add2f_83',
 u'add2f_11',
 u'add2f_668',
 u'add2f_239',
 u'add2f_662',
 u'add2f_534',
 u'add2f_2',
 u'add2f_446',
 u'add2

In [75]:
from sklearn.metrics import log_loss
res = []
for i in np.arange(0, 1.005, 0.01):
    q = ml.train.lr_s3 * i + ml.train.lgbm_s2 * (1-i)
    qual = log_loss( ml.train.cardio, q)
    res.append((i, qual))

In [ ]:
#importance = ml.EG.transformer_dict["xgb_s1"].transformer.booster().get_fscore()
#imp_fl = list(pd.Series(index = importance.keys(), data = importance.values()).sort_values(ascending = False).index)

In [ ]:
#ml.remove_step("add3f")
#ml.add_step("add3f", additive_featuring(imp_fl, 3, "cardio", lrclf_params_1, "add3f"), prerequisites = ["scale1"])
#ml.compute_step("add3f")

In [87]:
save_model(ml, "models/train2/")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))
xgboost.plot_importance(ml.EG.transformer_dict["xgb_s1"].transformer, ax = ax)

In [ ]:
std = ml.test[["xgb_s1", "id"]].groupby("id").std()

In [ ]:
measure_logloss(ml, "train", "cardio", "xgb_s1")

In [ ]:
#save with averaging
ml.test[["lr_s2", "id"]].groupby("id").mean().reset_index().drop("id", axis = 1).to_csv("lr_s2.csv", header = False, index = False)

In [ ]:
ml.test[["lgbm_s1", "id"]].groupby("id").mean().reset_index().drop("id", axis = 1).to_csv("lgbm_s1.csv", header = False, index = False)

In [81]:
def get_proba(d):
    d = d.astype(float)
    sum_v = sum(d.values)
    for i in d.keys():
        d[i] *= (1.0 / sum_v)
    return d

def fill_categorical_nans(train, test, features):
    train = train.copy()
    test = test.copy()
    for f in features:
        train_d = get_proba(train[f].value_counts())
        train.loc[train[f].isnull(), f] = np.random.choice(train_d.keys(), size = len(train[train[f].isnull()]), 
                                                           p=train_d.values)
        
        test.loc[test[f].isnull(), f] = np.random.choice(train_d.keys(), size = len(test[test[f].isnull()]), 
                                                           p=train_d.values)
    return train, test

In [82]:
ml.test["lc"] = ml.test.lgbm_s2 * 0.5 + ml.test.lr_s3 * 0.5

In [83]:
#save with nan filling
train, test = fill_categorical_nans(train, test, ["smoke", "alco", "active"])
out = pd.merge(test[["id", "smoke", "alco", "active"]], ml.test[["id", "smoke", "alco", "active", "lc"]], on = ["id", "smoke", "alco", "active"], how = "inner")
out.drop(["id", "smoke", "alco", "active"], axis = 1).to_csv("lc_nan.csv", header = False, index = False)